# Stability of the Grid System

> we’ll build a binary classification model to predict if a grid is stable or unstable using the UCI Electrical Grid Stability Simulated dataset.

- author: Victor Omondi
- toc: true
- comments: true
- categories: [machine-learning, classification]
- image: images/sgs-shield.png

# Libraries

In [17]:
import warnings

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use("ggplot")

from sklearn.model_selection import (train_test_split, 
                                     RandomizedSearchCV)
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import ExtraTreeClassifier

## Libraries Settings

In [2]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

# Overview

Electrical grids require a balance between electricity supply and demand in order to be stable. Conventional systems achieve this balance through demand-driven electricity production. For future grids with a high share of inflexible (i.e., renewable) energy source, the concept of demand response is a promising solution. This implies changes in electricity consumption in relation to electricity price changes. In this work, we’ll build a binary classification model to predict if a grid is stable or unstable using the UCI Electrical Grid Stability Simulated dataset {% fn 1 %}.

# Data Importation

In [4]:
elec = pd.read_csv("datasets/Data_for_UCI_named.csv")
elec.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


It has 12 primary predictive features and two dependent variables.

## Predictive features:

- `'tau1'` to `'tau4'`: the reaction time of each network participant, a real value within the range 0.5 to 10 (`'tau1'` corresponds to the supplier node, `'tau2'` to `'tau4'` to the consumer nodes);
- `'p1'` to `'p4'`: nominal power produced (positive) or consumed (negative) by each network participant, a real value within the range -2.0 to -0.5 for consumers (`'p2'` to `'p4'`). As the total power consumed equals the total power generated, $p1 (supplier node) = - (p2 + p3 + p4)$;
- `'g1'` to `'g4'`: price elasticity coefficient for each network participant, a real value within the range 0.05 to 1.00 (`'g1'` corresponds to the supplier node, `'g2'` to `'g4'` to the consumer nodes; `'g'` stands for `'gamma'`);

## Dependent variables:

- `'stab'`: the maximum real part of the characteristic differential equation root (if positive, the system is linearly unstable; if negative, linearly stable);
- `'stabf'`: a categorical (binary) label (`'stable'` or `'unstable'`).

# Data Cleaning

In [5]:
elec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   tau1    10000 non-null  float64
 1   tau2    10000 non-null  float64
 2   tau3    10000 non-null  float64
 3   tau4    10000 non-null  float64
 4   p1      10000 non-null  float64
 5   p2      10000 non-null  float64
 6   p3      10000 non-null  float64
 7   p4      10000 non-null  float64
 8   g1      10000 non-null  float64
 9   g2      10000 non-null  float64
 10  g3      10000 non-null  float64
 11  g4      10000 non-null  float64
 12  stab    10000 non-null  float64
 13  stabf   10000 non-null  object 
dtypes: float64(13), object(1)
memory usage: 1.1+ MB


In [6]:
elec.isnull().sum()

tau1     0
tau2     0
tau3     0
tau4     0
p1       0
p2       0
p3       0
p4       0
g1       0
g2       0
g3       0
g4       0
stab     0
stabf    0
dtype: int64

There is no missing value in the dataset. The dataset has 10000 rows and 14 columns.

Because of the direct relationship between 'stab' and 'stabf' ('stabf' = 'stable' if 'stab' <= 0, 'unstable' otherwise), 'stab' should be dropped and 'stabf' will remain as the sole dependent variable (binary classification).

In [8]:
elec.stabf.value_counts()

unstable    6380
stable      3620
Name: stabf, dtype: int64

In [9]:
elec[(elec.stab<=0) & (elec.stabf!='stable')]

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf


In [10]:
elec[(elec.stab>0) & (elec.stabf!='unstable')]

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf


All the rows are observing the direct relationship of stab and stabf

In [13]:
elec.drop(columns=['stab'], inplace=True)
elec.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,unstable


In [14]:
X = elec.drop(columns='stabf')
X.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923


In [15]:
y = elec.stabf
y.head()

0    unstable
1      stable
2    unstable
3    unstable
4    unstable
Name: stabf, dtype: object

# Data modeling

We will split the data into an 80-20 train-test split with a random state of “1”. And use the standard scaler to transform the train set (x_train, y_train) and the test set (x_test). We will use scikit learn to train a random forest and extra trees classifier. We'll also use xgboost and lightgbm to train an extreme boosting model and a light gradient boosting model. random_state = 1 for training all models and evaluate on the test set will be used. 

Also, to improve the Extra Trees Classifier, we will use the following parameters (number of estimators, minimum number of samples, minimum number of samples for leaf node and the number of features to consider when looking for the best split) for the hyperparameter grid needed to run a Randomized Cross Validation Search (RandomizedSearchCV). 

In [16]:
n_estimators = [50, 100, 300, 500, 1000]

min_samples_split = [2, 3, 5, 7, 9]

min_samples_leaf = [1, 2, 4, 6, 8]

max_features = ['auto', 'sqrt', 'log2', None] 

hyperparameter_grid = {'n_estimators': n_estimators,
                       'min_samples_leaf': min_samples_leaf,
                       'min_samples_split': min_samples_split,
                       'max_features': max_features}

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=1)
X_train_scaled = StandardScaler().fit_transform(X_train)
X_test_scaled = StandardScaler().fit_transform(X_test)

In [25]:
rf = RandomForestClassifier(random_state=1)
rf.fit(X_train_scaled, y_train)
rf_pred = rf.predict(X_test_scaled)
rf_pred[:5]

array(['unstable', 'unstable', 'stable', 'stable', 'unstable'],
      dtype=object)

In [26]:
et = ExtraTreeClassifier(random_state=1)
et.fit(X_train_scaled, y_train)
et_pred = et.predict(X_test_scaled)
et_pred[:5]

array(['unstable', 'unstable', 'stable', 'unstable', 'unstable'],
      dtype=object)

In [27]:
y_test[:5]

9953    unstable
3850    unstable
4962      stable
3886      stable
5437    unstable
Name: stabf, dtype: object

In [33]:
rs = RandomizedSearchCV(rf, param_distributions=hyperparameter_grid, cv=5)
rs.fit(X_train_scaled, y_train)
print(f"Best Score: {round(rs.best_score_, 2)}\nBest Params: {rs.best_params_}")

Best Score: 0.92
Best Params: {'n_estimators': 500, 'min_samples_split': 3, 'min_samples_leaf': 2, 'max_features': 'sqrt'}


In [34]:
hyperparameter_grid_2 = {'min_samples_leaf': min_samples_leaf,
                       'min_samples_split': min_samples_split,
                       'max_features': max_features}
rs_2 = RandomizedSearchCV(et, param_distributions=hyperparameter_grid_2, cv=5)
rs_2.fit(X_train_scaled, y_train)
print(f"Best Score: {round(rs_2.best_score_, 2)}\nBest Params: {rs_2.best_params_}")

Best Score: 0.83
Best Params: {'min_samples_split': 3, 'min_samples_leaf': 8, 'max_features': None}


{{'Dataset: https://archive.ics.uci.edu/ml/datasets/Electrical+Grid+Stability+Simulated+Data+' | fndetail: 1}}